<a href="https://colab.research.google.com/github/hBar2013/DS-Unit-2-Classification-1/blob/master/module1-logistic-regression/kim_lowry_logistic_regression_categorical_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Features

Your goal is to predict the operating condition of a waterpoint for each record in the dataset. You are provided the following set of information about the waterpoints:

- `amount_tsh` : Total static head (amount water available to waterpoint)
- `date_recorded` : The date the row was entered
- `funder` : Who funded the well
- `gps_height` : Altitude of the well
- `installer` : Organization that installed the well
- `longitude` : GPS coordinate
- `latitude` : GPS coordinate
- `wpt_name` : Name of the waterpoint if there is one
- `num_private` :  
- `basin` : Geographic water basin
- `subvillage` : Geographic location
- `region` : Geographic location
- `region_code` : Geographic location (coded)
- `district_code` : Geographic location (coded)
- `lga` : Geographic location
- `ward` : Geographic location
- `population` : Population around the well
- `public_meeting` : True/False
- `recorded_by` : Group entering this row of data
- `scheme_management` : Who operates the waterpoint
- `scheme_name` : Who operates the waterpoint
- `permit` : If the waterpoint is permitted
- `construction_year` : Year the waterpoint was constructed
- `extraction_type` : The kind of extraction the waterpoint uses
- `extraction_type_group` : The kind of extraction the waterpoint uses
- `extraction_type_class` : The kind of extraction the waterpoint uses
- `management` : How the waterpoint is managed
- `management_group` : How the waterpoint is managed
- `payment` : What the water costs
- `payment_type` : What the water costs
- `water_quality` : The quality of the water
- `quality_group` : The quality of the water
- `quantity` : The quantity of water
- `quantity_group` : The quantity of water
- `source` : The source of the water
- `source_type` : The source of the water
- `source_class` : The source of the water
- `waterpoint_type` : The kind of waterpoint
- `waterpoint_type_group` : The kind of waterpoint

### Labels

There are three possible values:

- `functional` : the waterpoint is operational and there are no repairs needed
- `functional needs repair` : the waterpoint is operational, but needs repairs
- `non functional` : the waterpoint is not operational

In [69]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv


In [7]:
!pip install category_encoders

     |████████████████████████████████| 92kB 5.3MB/s 


In [0]:
import category_encoders as ce

In [0]:
import pandas as pd

In [0]:
train_features_file = 'train_features.csv'
train_labels_file = 'train_labels.csv'
test_feat_file = 'test_features.csv'

In [0]:
train_feat = pd.read_csv(train_features_file)
train_labels = pd.read_csv(train_labels_file)
test_feat = pd.read_csv(test_features_file)

In [76]:
train_feat.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [34]:
train_feat.query('population == 0').population.count()

21381

In [85]:
train_feat.describe()


,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [0]:
train_feat.describe(exclude='number').T

In [0]:
train_feat['extraction_type'].value_counts(dropna=False)

In [0]:
train_feat['extraction_type_group'].value_counts(dropna=False)

In [130]:
train_feat['extraction_type_class'].value_counts(dropna=False)

gravity         26780
handpump        16456
other            6430
submersible      6179
motorpump        2987
rope pump         451
wind-powered      117
Name: extraction_type_class, dtype: int64

In [0]:
train_merged = train_feat.merge(train_label)

In [129]:
train_merged.groupby('extraction_type_class')['status_group'].value_counts(normalize=True)

extraction_type_class  status_group           
gravity                functional                 0.599253
                       non functional             0.299888
                       functional needs repair    0.100859
handpump               functional                 0.630469
                       non functional             0.309067
                       functional needs repair    0.060464
motorpump              non functional             0.573820
                       functional                 0.379980
                       functional needs repair    0.046200
other                  non functional             0.807932
                       functional                 0.160031
                       functional needs repair    0.032037
rope pump              functional                 0.649667
                       non functional             0.312639
                       functional needs repair    0.037694
submersible            functional                 0.538760
         

In [128]:
train_merged.groupby('payment_type')['status_group'].value_counts(normalize=True)


payment_type  status_group           
annually      functional                 0.752334
              non functional             0.179846
              functional needs repair    0.067820
monthly       functional                 0.660482
              non functional             0.227831
              functional needs repair    0.111687
never pay     non functional             0.475856
              functional                 0.448911
              functional needs repair    0.075233
on failure    functional                 0.620593
              non functional             0.308636
              functional needs repair    0.070772
other         functional                 0.579696
              non functional             0.308349
              functional needs repair    0.111954
per bucket    functional                 0.677796
              non functional             0.276683
              functional needs repair    0.045520
unknown       non functional             0.514527
            

In [133]:
train_merged.groupby('waterpoint_type_group')['status_group'].value_counts(normalize=True)

waterpoint_type_group  status_group           
cattle trough          functional                 0.724138
                       non functional             0.258621
                       functional needs repair    0.017241
communal standpipe     functional                 0.576491
                       non functional             0.339523
                       functional needs repair    0.083986
dam                    functional                 0.857143
                       non functional             0.142857
hand pump              functional                 0.617852
                       non functional             0.323307
                       functional needs repair    0.058840
improved spring        functional                 0.718112
                       non functional             0.173469
                       functional needs repair    0.108418
other                  non functional             0.822414
                       functional                 0.131661
         

In [0]:
train_merged.groupby('source_type')['status_group'].value_counts(normalize=True)

In [127]:
train_merged.groupby('basin')['status_group'].value_counts(normalize=True)


basin                    status_group           
Internal                 functional                 0.575723
                         non functional             0.352730
                         functional needs repair    0.071548
Lake Nyasa               functional                 0.653687
                         non functional             0.297148
                         functional needs repair    0.049164
Lake Rukwa               non functional             0.482478
                         functional                 0.407498
                         functional needs repair    0.110024
Lake Tanganyika          functional                 0.483053
                         non functional             0.401586
                         functional needs repair    0.115361
Lake Victoria            functional                 0.497658
                         non functional             0.405835
                         functional needs repair    0.096507
Pangani                  functional 

In [131]:
train_merged.groupby('management_group')['status_group'].value_counts(normalize=True)


management_group  status_group           
commercial        functional                 0.614349
                  non functional             0.353491
                  functional needs repair    0.032161
other             functional                 0.559915
                  non functional             0.380700
                  functional needs repair    0.059385
parastatal        functional                 0.576923
                  non functional             0.303733
                  functional needs repair    0.119344
unknown           non functional             0.552585
                  functional                 0.399287
                  functional needs repair    0.048128
user-group        functional                 0.538236
                  non functional             0.387350
                  functional needs repair    0.074414
Name: status_group, dtype: float64

In [132]:
train_merged.groupby('quality_group')['status_group'].value_counts(normalize=True)


quality_group  status_group           
colored        functional                 0.502041
               non functional             0.387755
               functional needs repair    0.110204
fluoride       functional                 0.723502
               non functional             0.216590
               functional needs repair    0.059908
good           functional                 0.565941
               non functional             0.357236
               functional needs repair    0.076823
milky          functional                 0.544776
               non functional             0.437811
               functional needs repair    0.017413
salty          non functional             0.482002
               functional                 0.460828
               functional needs repair    0.057170
unknown        non functional             0.840618
               functional                 0.140725
               functional needs repair    0.018657
Name: status_group, dtype: float64

In [0]:
train_merged.groupby('construction_year')['status_group'].value_counts(normalize=True)


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [148]:
X_train = train_feat
y_train = train_labels['status_group']

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size=0.80, test_size=0.20, 
    stratify=y_train, random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((47520, 40), (11880, 40), (47520,), (11880,))

In [149]:
num_features = ['region_code', 'district_code']
               
cat_features = ['payment_type','waterpoint_type', 'quantity', 
                'extraction_type', 'permit', 'management'] 
#             ['payment_type', 'waterpoint_type', 'quantity',
#               'extraction_type', 'permit', 'management'] 
               
          
#                'waterpoint_type_group', 'quantity', 'management_group']

features = cat_features + num_features

X_train_subset = X_train[features]
X_val_subset = X_val[features]

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train_subset)
X_val_encoded = encoder.transform(X_val_subset)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)

model = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)
model.fit(X_train_scaled, y_train)
print('Validation Accuracy', model.score(X_val_scaled, y_val))

Validation Accuracy 0.7223905723905724


In [0]:
X_test_subset = test_feat[features]
X_test_encoded = encoder.transform(X_test_subset)
X_test_scaled = scaler.transform(X_test_encoded)

In [123]:
all(X_test_encoded.columns == X_train_encoded.columns)

True

In [67]:
X_test_encoded.columns, X_train_encoded.columns

(Index(['payment_type_never pay', 'payment_type_monthly',
        'payment_type_per bucket', 'payment_type_unknown',
        'payment_type_annually', 'payment_type_on failure',
        'payment_type_other', 'waterpoint_type_group_communal standpipe',
        'waterpoint_type_group_hand pump', 'waterpoint_type_group_other',
        'waterpoint_type_group_improved spring',
        'waterpoint_type_group_cattle trough', 'waterpoint_type_group_dam',
        'quantity_insufficient', 'quantity_enough', 'quantity_dry',
        'quantity_seasonal', 'quantity_unknown', 'region_code', 'gps_height'],
       dtype='object'),
 Index(['payment_type_never pay', 'payment_type_monthly',
        'payment_type_per bucket', 'payment_type_unknown',
        'payment_type_annually', 'payment_type_on failure',
        'payment_type_other', 'waterpoint_type_group_communal standpipe',
        'waterpoint_type_group_hand pump', 'waterpoint_type_group_other',
        'waterpoint_type_group_improved spring',
     

In [0]:
sumbission_sample_file = 'sample_submission.csv'
sample_submission = pd.read_csv(sumbission_sample_file)

In [0]:
y_pred = model.predict(X_test_scaled)
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('kel-submission-02.csv', index=False)

In [125]:
!head kel-submission-01.csv

id,status_group
50785,non functional
51630,functional
17168,non functional
45559,non functional
49871,functional
52449,functional
24806,functional
28965,non functional
36301,functional


In [0]:
from google.colab import files
files.download('kel-submission-02.csv')

## Submit to predictive modeling competition


### Write submission CSV file

The format for the submission file is simply the row id and the predicted label (for an example, see `sample_submission.csv` on the data download page.

For example, if you just predicted that all the waterpoints were functional you would have the following predictions:

<pre>id,status_group
50785,functional
51630,functional
17168,functional
45559,functional
49871,functional
</pre>

Your code to generate a submission file may look like this: 
<pre># estimator is your scikit-learn estimator, which you've fit on X_train

# X_test is your pandas dataframe or numpy array, 
# with the same number of rows, in the same order, as test_features.csv, 
# and the same number of columns, in the same order, as X_train

y_pred = estimator.predict(X_test)


# Makes a dataframe with two columns, id and status_group, 
# and writes to a csv file, without the index

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('your-submission-filename.csv', index=False)
</pre>

### Send submission CSV file to Kaggle

#### Option 1. Kaggle web UI
 
Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file.


#### Option 2. Kaggle API

Use the Kaggle API to upload your CSV file.

# Assignment
- Learn about the mathematics of Logistic Regression by watching Aaron Gallant's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes).
- Start a clean notebook.
- Do train/validate/test split with the Tanzania Waterpumps data.
- Begin to explore and clean the data. For ideas, refer to [The Quartz guide to bad data](https://github.com/Quartz/bad-data-guide),  a "reference to problems seen in real-world data along with suggestions on how to resolve them." One of the issues is ["Zeros replace missing values."](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values)
- Select different numeric and categorical features. 
- Do one-hot encoding. (Remember it may not work with high cardinality categoricals.)
- Scale features.
- Use scikit-learn for logistic regression.
- Get your validation accuracy score.
- Get and plot your coefficients.
- Submit your predictions to our Kaggle competition.
- Commit your notebook to your fork of the GitHub repo.

## Stretch Goals
- Begin to visualize the data.
- Try different [scikit-learn scalers](https://scikit-learn.org/stable/modules/preprocessing.html)
- Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html):

> Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:

> - **Convenience and encapsulation.** You only have to call fit and predict once on your data to fit a whole sequence of estimators.
> - **Joint parameter selection.** You can grid search over parameters of all estimators in the pipeline at once.
> - **Safety.** Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.
